# Newman Watts Strogatz Graph
Note that it is just an exmple, and the scenarios used in the thesis differ from the following.

In [31]:
import mesa
import numpy as np
import random
import networkx as nx 
import bisect


class MarriageModel(mesa.Model):
    def __init__(self, num_nodes = 50, avg_node_degree=10):

        self.num_nodes = num_nodes
        # prob = avg_node_degree / self.num_nodes
        self.G = nx.newman_watts_strogatz_graph(n=self.num_nodes, k=avg_node_degree, p=0.2)
        self.grid = mesa.space.NetworkGrid(self.G)
        self.schedule = mesa.time.RandomActivation(self)
      
        # # Determine the node for initial agents
        # list_of_possible_nodes = self.random.sample(list(self.G), self.num_nodes) 
        # # Create agents
        # for i in range(self.num_nodes):
        #     # Add the agent to a random grid cell
        #     agent = MarriageAgent(i, self)
        #     self.schedule.add(agent)
        #     self.grid.place_agent(agent, list_of_possible_nodes[i])

        for node in self.G.nodes: 
            agent = MarriageAgent(node, self)
            self.schedule.add(agent)
            self.grid.place_agent(agent, node)
        
        self.datacollector = mesa.DataCollector(
                agent_reporters={
                                "Age" : lambda a: a.age,
                                "Education": lambda a: a.education,
                                "Gender": lambda a: a.gender,
                                "Income": lambda a: round(a.income, 3),
                                "Spouse": lambda a: a.spouse.unique_id if a.spouse else None,
                                "Parents": lambda a: a.parents_id,
                                "Generation": lambda a: a.generation,
                                "Cohort": lambda a: a.cohort,
                                "Children": lambda a: a.children
                })
        
        self.running = True
        self.datacollector.collect(self)
        
        self.last_child_id = self.num_nodes-1
    
    def step(self):
        # Collect data
        self.datacollector.collect(self)
        # Advance the model by one step
        self.schedule.step()
    
    def run_model(self, n):
        for i in range(n):
            self.step()
        

class MarriageAgent(mesa.Agent):
    """ Agents with an educational level, age, gender, and income """

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.spouse = None
        self.parents_id = []
        self.age = random.randint(16, 59) 
        self.children = []
        self.cohort = MarriageAgent.get_cohort(self)
        self.gender = np.random.choice(["M", "F"], p=[0.5, 0.5]) 
        self.generation = 1
        self.education = MarriageAgent.get_education(self)
        self.income = random.uniform(0, 800)
        

        
    
    def get_cohort(self):
        if self.age > 40:
            return 1
        elif self.age > 20:
            return 2
        else:
            return 3
    
    
# To-do: Make children's education dependent on parents'
    def get_education(self):
        isced = list(range(1,6))  
        # 1 = Less than primary education; 2 = Primary education; 3 = Lower secondary education; 
        # 4 = Upper secondary or post-secondary non-tertiary education
        # 5 = Tertiary education
        if self.gender == 'M' :
            if self.cohort == 1:
                return np.random.choice(isced, p = [0.20, 0.35, 0.25, 0.15, 0.05]) 
            elif self.cohort == 2: 
                return np.random.choice(isced, p = [0.15, 0.30, 0.30, 0.20, 0.05])
            elif self.cohort ==3:
                return np.random.choice(isced, p = [0.10, 0.30, 0.25, 0.25, 0.10])
        else :
            if self.cohort == 1:
                return np.random.choice(isced, p = [0.39, 0.35, 0.15, 0.10, 0.01]) 
            elif self.cohort == 2: 
                return np.random.choice(isced, p = [0.20, 0.35, 0.30, 0.10, 0.05])
            elif self.cohort ==3:
                return np.random.choice(isced, p = [0.10, 0.30, 0.25, 0.25, 0.10])
            
        
    
    def get_spouse(self):
        
        adjacent_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        potential_partners = [
                agent
                for agent in self.model.grid.get_cell_list_contents(adjacent_nodes)
                if agent not in self.children and agent.spouse is None and agent.gender != self.gender 
                and ((not agent.parents_id and not self.parents_id) or (set(self.parents_id) != set(agent.parents_id)))] 
        
        # 90% Homogamy        
        for p in potential_partners:
            if self.random.random() < 0.9:
                potential_partners = [p for p in potential_partners if p.education == self.education and self.cohort-1 <= p.cohort <= self.cohort+1]
            else:
                potential_partners = [p for p in potential_partners if p.education != self.education and self.cohort-1 <= p.cohort <= self.cohort+1]

        # Choose a spouse from potential partners
        if len(potential_partners) > 0:
                spouse = random.choice(potential_partners)
                self.spouse = spouse
                spouse.spouse = self
            
    def reproduce(self):
        
        
        # Place the new agent in an empty cell
        empty_cells = [cell for cell in self.model.grid.get_neighbors(self.pos,  include_center=False) if self.model.grid.is_cell_empty(cell)]
        if len(empty_cells) > 0:
            child = MarriageAgent(self.model.last_child_id + 1, self.model)
            self.model.last_child_id += 1
            new_pos = random.choice(empty_cells)
            self.model.grid.place_agent(child, new_pos)
            self.model.schedule.add(child)
            self.children.append(child.unique_id)
            self.spouse.children.append(child.unique_id)
            child.parents_id = self.unique_id, self.spouse.unique_id
            child.generation = max(self.generation, self.spouse.generation)+1
            child.cohort = max(self.cohort, self.spouse.cohort)+1

#         else:
#             newnode = child.unique_id
#             self.model.G.add_node(newnode)
#             self.model.grid.place_agent(child, newnode)
#             self.model.schedule.add(child)
  

    def get_cohort_list(self):
        cohorts = [agent for agent in model.grid.get_all_cell_contents() ] #if self.step == agent.step 
        value = []
        for c in cohorts:
            cohort_num = c.cohort
            value.append(cohort_num)
        return value
            
    
    def step(self):
        # Increment age
        self.age +=1
        
        #First, seek a partner if self.spouse == None.
        if self.spouse is None:
            self.get_spouse()
            
        # Second, if a partner exists, reproduce (randomly).    
        else:
            if len(self.children) < 2 and self.gender == 'F' and self.age < 45:
                self.model.num_nodes += 1
                self.reproduce()
       
        if self.cohort == min(self.get_cohort_list()):
            if self.random.random() < 0.4:
                self.model.grid.remove_agent(self)
                self.model.schedule.remove(self)

In [32]:
model = MarriageModel(50, 10)
for i in range(10):
    model.step()

In [33]:
import pandas as pd

agentinfo = model.datacollector.get_agent_vars_dataframe()
agentinfo[pd.isnull(agentinfo['Parents']) == False]

Age  Education Gender   Income  Spouse   Parents  Generation  \
Step AgentID                                                                 
0    0         43          2      M  767.229     NaN        []           1   
     1         19          2      F  179.268     NaN        []           1   
     2         43          1      F    1.875     NaN        []           1   
     3         48          1      M  239.391     NaN        []           1   
     4         27          2      F  662.229     NaN        []           1   
...           ...        ...    ...      ...     ...       ...         ...   
9    66        57          4      M   98.896     NaN    (4, 8)           2   
     67        47          2      F  239.043    56.0   (47, 0)           2   
     68        43          2      F  367.013    64.0   (9, 37)           2   
     69        46          3      M  616.544     NaN  (68, 64)           3   
     70        42          2      M  696.918     NaN  (68, 64)           3   

              Cohort  Children  
Step AgentID                    
0    0             1  [61, 67]  
     1             3  [56, 64]  
     2             1      [50]  
     3             1        []  
     4             2  [59, 66]  
...              ...       ...  
9    66            3        []  
     67            3        []  
     68            3  [69, 70]  
     69            5        []  
     70            5        []  

[428 rows x 9 columns]